In [1]:
from __future__ import print_function
import os
import sys
import time
import numpy as np
import re
import faiss
import inspect
from multiprocessing.dummy import Pool as ThreadPool
from matplotlib import pyplot

In [12]:
"""
Some useful wiki:

Getting started: https://github.com/facebookresearch/faiss/wiki/Getting-started

Index factory: https://github.com/facebookresearch/faiss/wiki/The-index-factory
    e.g., index = index_factory(128, "OPQ16_64,IMI2x8,PQ8+16")
    
OPQ / PCA transition: https://github.com/facebookresearch/faiss/wiki/Pre--and-post-processing

C++ API (same as python): https://faiss.ai/cpp_api/struct/structfaiss_1_1MultiIndexQuantizer.html

"""

In [6]:
m = int(16)
d = int(128)                          # dimension
nb = int(1e6)                     # database size
nq = int(1e4)                     # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [13]:
opq = faiss.OPQMatrix(d, m)
opq.train(xb)

In [20]:
opq

<faiss.swigfaiss_avx2.OPQMatrix; proxy of <Swig Object of type 'faiss::OPQMatrix *' at 0x7f3efab92a80> >

In [25]:
# index = faiss.index_factory(d, "OPQ16,IMI2x8,PQ{}".format(m))
# MultiIndexQuantizer(int d, size_t M, size_t nbits)
index = faiss.MultiIndexQuantizer(d, m, 8)

In [26]:
index = faiss.IndexPreTransform (opq, index)

In [27]:
index.add(xb)                  # add vectors to the index

RuntimeError: Error in virtual void faiss::IndexPreTransform::add(faiss::Index::idx_t, const float*) at IndexPreTransform.cpp:162: Error: 'is_trained' failed